In [ ]:
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pickle
import torch

max_training_samples = 20000

# load test dtaset
file_name = '/content/drive/MyDrive/refCOCOg Visual Grounding/yolov8x+clip/yolo_v8x_1_dictionary_full_train.p'
with open(file_name, 'rb') as f:
    data_train = pickle.load(f)[:max_training_samples]

# load test dataset
# file_name = '/content/drive/MyDrive/refCOCOg Visual Grounding/yolov8x+clip/yolo_v8x_1_dictionary_full_test.p'
# with open(file_name, 'rb') as f:
#     data_test = pickle.load(f)

# load test dataset
file_name = '/content/drive/MyDrive/refCOCOg Visual Grounding/yolov8x+clip/yolo_v8x_1_dictionary_full_val.p'
with open(file_name, 'rb') as f:
    data_val = pickle.load(f)


# load the dataset


from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm


# load the dataset

from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

def get_data_single_prompt(full_data):

    text_encoding, cat_encoding_text, box_coords, target_boxes = [], [], [], []

    for idx in tqdm(list(full_data)):
        # for _ in range(data['image_emb'].shape[0]):
        for idx_text in range(full_data[idx]['text_emb'].shape[0]):

            # number of available crops
            number_of_crop = min(full_data[idx]['image_emb'].shape[0], len(full_data[idx]['df_boxes']))

            if number_of_crop == 0:
                break

            text_enc = full_data[idx]['text_emb'][idx_text].unsqueeze(1)
            box_enc = full_data[idx]['image_emb'][:number_of_crop,:].permute(1, 0)

            # shape: (number of samples, 512, 1)
            text_encoding.append(text_enc)

            # shape: (number of samples, 512, number of crop embeddings)
            # box_encoding.append(box_enc)

            # shape: (number of samples, number of boxes, 4)
            box_coords.append(torch.stack([torch.tensor(full_data[idx]['df_boxes'].iloc[i][:4]).type(torch.float16)
                                                    for i in range(number_of_crop)]))

            # shape: torch.Size(number of samples, 2, 512, number of boxes])
            cat_encoding_text.append(torch.cat([box_enc.unsqueeze(0), torch.stack([text_enc.squeeze(1) for _ in range(number_of_crop)], dim=1).unsqueeze(0)], dim=0))

            # shape: (number of samples, 1, 4)
            target_boxes.append(torch.tensor(full_data[idx]['bbox_target']).type(torch.float16).unsqueeze(0))

           # (number of samples, 1, 512, 1)          padd the tensors to stack them togheter
    return torch.stack(text_encoding).unsqueeze(1), torch.stack([torch.nn.functional.pad(b.permute(0, 2, 1), (0, 0, 0, 48 - b.shape[2])).permute(0, 2, 1) for b in cat_encoding_text]), torch.stack([torch.nn.functional.pad(b, (0, 0, 0, 48 - b.shape[0])) for b in box_coords]), torch.stack(target_boxes)



# text_encoding_test, box_encoding_test, box_coords_test, target_boxes_test = get_data_single_prompt(data_test)
text_encoding_val, box_encoding_val, box_coords_val, target_boxes_val = get_data_single_prompt(data_val)
del data_val
text_encoding_train, box_encoding_train, box_coords_train, target_boxes_train = get_data_single_prompt(data_train)

del data_train

# text_encoding_train, box_encoding_train, box_coords_train, target_boxes_train = text_encoding_train[:20000], box_encoding_train[:20000], box_coords_train[:20000], target_boxes_train[:20000]

def box_norm_rescale(box_target):
    """ Rescale the box_target
    Args:
        box_target: (number of samples, 1, 4)

    Returns:
        box_target: (number of samples, 1, 4)

    """
    # convert the box_pred to x1, y1, x2, y2
    box_target[:, 0, 2] = box_target[:, 0, 0] + box_target[:, 0, 2]
    box_target[:, 0, 3] = box_target[:, 0, 1] + box_target[:, 0, 3]

    return box_target

# box rescaling
# target_boxes_test = box_norm_rescale(target_boxes_test)
target_boxes_val = box_norm_rescale(target_boxes_val)
target_boxes_train = box_norm_rescale(target_boxes_train)


# print(text_encoding_test.shape)
# print(box_encoding_test.shape)
# print(box_coords_test.shape)
# print(target_boxes_test.shape)

# target_boxes_test = target_boxes_test.squeeze(1)
target_boxes_val = target_boxes_val.squeeze(1)
target_boxes_train = target_boxes_train.squeeze(1)


100%|██████████| 2573/2573 [00:00<00:00, 620976.13it/s]


RuntimeError: ignored

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch_lightning as pl
import torchmetrics
import torchvision

#############################################################
# with multiple textual prompts
#############################################################

############################ 1. MLP

from typing import Any, Optional


from pytorch_lightning.utilities.types import STEP_OUTPUT


class MLP(nn.Module):
    def __init__(self, out_dim=512):
        super().__init__()
        self.l1 = nn.Linear(4, 64)
        self.l2 = nn.Linear(64, 32)
        self.l3 = nn.Linear(32, out_dim)
        self.gelu = nn.GELU()

    def forward(self, x):
        x = self.gelu(self.l1(x))
        x = self.gelu(self.l2(x))
        return self.l3(x)


############################ 2. BoxEncoder

class LinearBoxWeightingBlock(nn.Module):
    def __init__(self, hidden_dim=64, n_head=4, number_transformer_layers=3, dropout=0.1):
        super().__init__()

        self.linear_1 = nn.Linear(3, hidden_dim)
        self.layernorm_1 = nn.LayerNorm(hidden_dim)
        self.gelu = nn.GELU()

        # In our architecture the transformer encoder has the layer norm first.
        # According with the results of the paper 'On Layer Normalization in 
        # the Transformer Architecture' the convergence is faster with the layer 
        # norm first.
        # source: https://arxiv.org/pdf/2002.04745.pdf
        self.transformer_encoder = nn.TransformerEncoder(
                nn.TransformerEncoderLayer(d_model=3, nhead=n_head, norm_first=True), num_layers=number_transformer_layers
            )

        # This layer is used to combine the output of the transformer encoder
        # with the initial concatenated embedding. The result of this layer is
        # to get a single vector for each box.
        self.bilinear_sum = nn.Bilinear(hidden_dim, hidden_dim, 1)

        self.dropout = nn.Dropout(dropout)


    def forward(self, x, box, text_emb):
        ##################################################   BOX ENCODER BLOCK
        # x: (batch_size, 4, n_boxes) -> (batch_size, 1, 512, n_boxes)
        box_emb = self.mlp(box).permute(0, 2, 1).unsqueeze(1)

        # x: (batch_size, 3, 512, n_boxes)
        x = torch.cat([x, box_emb], dim=1)              # concat EMB and box
        # del box_emb                                     # del box_emb

        ##########################################   WEIGHTED COMBINATION BLOCK
        # x: (batch_size, n_boxes, 512, 3)
        x_1 = x.permute(0, 3, 2, 1)                 # permute
        x = self.linear_1(x_1)                         # linear transformation
        x = self.gelu(x)                            # non-linearity
        x = self.layernorm_1(x)                     # add & norm
        x = self.transformer_encoder(x)             # transformer encoder

        # here we apply dropout only during training
        # over the embedding in output of the transformer 
        # encoder (regularyzation) ADD & DROP
        x = self.bilinear_sum(
            x + F.dropout(x, self.dropout, training=self.training), 
            x_1
        )                                           # get a weight for each box
        x = self.gelu(x)                            # non-linearity
        x = self.layernorm_2(x)                     # norm


        # x: torch.Size([BATCH_SIZE, N_BOXES, 512, 1]) -> torch.Size([BATCH_SIZE, N_BOXES, 1, 512])
        x = x.permute(0, 1, 3, 2)                   # permute

        return x, box_emb
    
class CLIPscoreBlock(nn.Module):
        ''' This block compute the bached clip score for each box.
            The input is the text embedding and the box embedding.
            The output is a tensor of shape (batch_size, n_boxes, 1)
            where each element is the score of the box.
        
        Args:
            @params text_emb: tensor of shape (batch_size, 512)
            @params box_emb: tensor of shape (batch_size, n_boxes, 512)

        Returns:
            @params returns x: tensor of shape (batch_size, n_boxes, 1)
        
        x:  [tex_0 @ box_1.T, tex_0 @ box_2.T, ..., tex_0 @ box_j]
            [tex_2 @ box_1.T, tex_2 @ box_2.T, ..., tex_2 @ box_j]
            [tex_3 @ box_1.T, tex_3 @ box_2.T, ..., tex_3 @ box_j]
                                    ...
                                    ...
            [tex_i @ box_1.T, tex_i @ box_2.T, ..., tex_i @ box_j]

        I am normalizing over the rows and summing over the columns
        to get the overall score for each box.

        x summarize the matching in between the sentence and the boxes
        x: [sum(tex_[:] @ box_1.T), sum(tex_[:] @ box_2.T), ..., sum(tex_[:] @ box_j)]
    '''
    def __init__(self, n_box=48):
        super().__init__()

        self.n_box = n_box


    def forward(self, box_encoding, text_emb):

        # torch.Size([256, 1, 1, 512]), torch.Size([256, 48, 1, 512])
        # torch.Size([256, 1, 512]), torch.Size([256, 48, 512]) 
        # torch.Size([256, 1, 512]) @ torch.Size([256, 512, 48]) ->                     # torch.Size([256, 10, 48])
        x = torch.bmm(text_emb.squeeze(2), box_encoding.squeeze(1).permute(0, 2, 1))
        # I should sum over the row to get the overall score for each box
        # specifically, I am summing the probabilities of each box being
        # the target box according to the matching score of the text encoding
        x = x.sum(dim=-2)                                                               # torch.Size([64, 10, 10]) -> torch.Size([64, 10])
        x_score = x_score.unsqueeze(2)                                                  # box: (batch_size, n_boxes) -> (batch_size, n_boxes, 1)

        return x




class BoxEncoder(pl.LightningModule):
    def __init__(self, out_dim_box=512, latent_dim=240, hidden_dim_regressor=256):
        super().__init__()

        self.linWeighting = LinearBoxWeightingBlock()
        self.clipScoreBlock = CLIPscoreBlock()

        self.box_regressor = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim_regressor),
            nn.GELU(),
            nn.LayerNorm(hidden_dim_regressor),
            nn.Linear(hidden_dim_regressor, hidden_dim_regressor//2),
            nn.GELU(),
            nn.LayerNorm(hidden_dim_regressor//2),
            nn.Linear(hidden_dim_regressor//2, 4)
        )


    def forward(self, x, box, text_emb):

        x, box_emb = self.linWeighting(x, box, text_emb)


        x = self.clipScoreBlock(x, text_emb)

        # box: (batch_size, 4) -> (batch_size, 5)
        box = torch.cat([x, box_emb], dim=-1)


        # box: (batch_size, n_boxes, 512) -> (batch_size, 4)
        box =  self.box_regressor(box.reshape(box.shape[0], box.shape[1]*box.shape[2]))

        return box

    def training_step(self, batch, batch_idx):
        cat_emb_text, box, t_emb, target = batch
        # print(cat_emb_text.shape, box.shape, y.shape)
        pred = self(cat_emb_text, box, t_emb)
        # https://arxiv.org/pdf/2108.12627
        # loss = F.huber_loss(y_hat, y)
        # https://arxiv.org/abs/1911.08287
        # loss = torchvision.ops.distance_box_iou_loss(y_hat, y)
        # https://arxiv.org/abs/1902.09630
        # loss = torchvision.ops.generalized_box_iou_loss(y_hat, y)

        mse_loss = self.MSE(pred, target)

        mae_loss = self.MAE(pred, target)

        huber_loss = self.HUBER(pred, target)

        g_box_iou_loss = self.generalized_box_iou_loss(pred, target, reduction = 'mean')

        d_box_iou_loss = self.distance_box_iou_loss(pred, target, reduction = 'mean')

        c_box_iou_loss = self.complete_box_iou_loss(pred, target, reduction = 'mean')

        self.log('val_mse_loss', mse_loss, on_step = True, on_epoch = True, prog_bar = True, logger = True)
        self.log('val_mae_loss', mae_loss, on_step = True, on_epoch = True, prog_bar = True, logger = True)
        self.log('val_huber_loss', huber_loss, on_step = True, on_epoch = True, prog_bar = True, logger = True)
        self.log('val_g_box_iou_loss', g_box_iou_loss, on_step = True, on_epoch = True, prog_bar = True, logger = True)
        self.log('val_d_box_iou_loss', d_box_iou_loss, on_step = True, on_epoch = True, prog_bar = True, logger = True)
        self.log('val_c_box_iou_loss', c_box_iou_loss, on_step = True, on_epoch = True, prog_bar = True, logger = True)

        return huber_loss

    def validation_step(self, batch, batch_idx):
        cat_emb_text, box, t_emb, target = batch
        # print(cat_emb_text.shape, box.shape, y.shape)
        pred = self(cat_emb_text, box, t_emb)
        # https://arxiv.org/pdf/2108.12627
        # loss = F.huber_loss(y_hat, y)
        # https://arxiv.org/abs/1911.08287
        # loss = torchvision.ops.distance_box_iou_loss(y_hat, y)
        # https://arxiv.org/abs/1902.09630
        # loss = torchvision.ops.generalized_box_iou_loss(y_hat, y)

        mse_loss = self.MSE(pred, target)

        mae_loss = self.MAE(pred, target)

        huber_loss = self.HUBER(pred, target)

        g_box_iou_loss = self.generalized_box_iou_loss(pred, target, reduction = 'mean')

        d_box_iou_loss = self.distance_box_iou_loss(pred, target, reduction = 'mean')

        c_box_iou_loss = self.complete_box_iou_loss(pred, target, reduction = 'mean')

        self.log('val_mse_loss', mse_loss, on_step = True, on_epoch = True, prog_bar = True, logger = True)
        self.log('val_mae_loss', mae_loss, on_step = True, on_epoch = True, prog_bar = True, logger = True)
        self.log('val_huber_loss', huber_loss, on_step = True, on_epoch = True, prog_bar = True, logger = True)
        self.log('val_g_box_iou_loss', g_box_iou_loss, on_step = True, on_epoch = True, prog_bar = True, logger = True)
        self.log('val_d_box_iou_loss', d_box_iou_loss, on_step = True, on_epoch = True, prog_bar = True, logger = True)
        self.log('val_c_box_iou_loss', c_box_iou_loss, on_step = True, on_epoch = True, prog_bar = True, logger = True)

        return huber_loss


    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=0.05)
        # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10)
        return [optimizer], [scheduler]


In [ ]:
# dataloaders

# architecture number 4

from torch.utils.data import DataLoader, TensorDataset

batch_size = 128

# dataset_test = TensorDataset(box_encoding_test.type(torch.float16), box_coords_test.type(torch.float16), text_encoding_test.type(torch.float16).permute(0, 1, 3, 2), target_boxes_test.type(torch.float16))
# dataset_val = TensorDataset(box_encoding_val.type(torch.float16), box_coords_val.type(torch.float16), text_encoding_val.type(torch.float16).permute(0, 1, 3, 2), target_boxes_val.type(torch.float16))

# dataset_test = TensorDataset(box_encoding_test.type(torch.float32), box_coords_test.type(torch.float32), text_encoding_test.type(torch.float32).permute(0, 1, 3, 2), target_boxes_test.type(torch.float32))

dataset_val = TensorDataset(box_encoding_val.type(torch.float32), box_coords_val.type(torch.float32), text_encoding_val.type(torch.float32).permute(0, 1, 3, 2), target_boxes_val.type(torch.float32))
dataset_train = TensorDataset(text_encoding_train.type(torch.float32), box_encoding_train.type(torch.float32), box_coords_train.type(torch.float32), target_boxes_train.type(torch.float32))

test_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)
# train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)


# most basic trainer, uses good defaults
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=0.00, patience=3, verbose=True, mode="min")
checkpoint_callback = ModelCheckpoint(dirpath='/content/drive/MyDrive/Colab Notebooks/dl')


# train the model
model = BoxEncoder().cuda()

print(model)

# print number of parameters
print(f"Number of parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)/1000000.0} M")

trainer = pl.Trainer(accelerator='auto', max_epochs=5, callbacks=[early_stop_callback, checkpoint_callback])

# train the model
trainer.fit(model, test_loader, val_loader)


RuntimeError: ignored

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

batch_size = 64

dataset_val = TensorDataset(box_encoding_val.type(torch.float32), box_coords_val.type(torch.float32), text_encoding_val.type(torch.float32).permute(0, 1, 3, 2), target_boxes_val.type(torch.float32))
dataset_train = TensorDataset(text_encoding_train.type(torch.float32), box_encoding_train.type(torch.float32), box_coords_train.type(torch.float32), target_boxes_train.type(torch.float32))

test_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)


# most basic trainer, uses good defaults
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=0.00, patience=10, verbose=True, mode="min")
checkpoint_callback = ModelCheckpoint(dirpath='/content/drive/MyDrive/Colab Notebooks/dl/', monitor='val_loss',save_top_k=2)


# train the model
model = BoxEncoder().cuda()

print(model)


# print number of parameters
print(f"Number of parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)/1000000.0} M")

trainer = pl.Trainer(accelerator='auto', max_epochs=20, callbacks=[early_stop_callback, checkpoint_callback])

# train the model
trainer.fit(model, test_loader, val_loader)